In [1]:
from Dependencies.random_circuit_generator import *
from Dependencies.functions_list import *
import random, psutil, time, numpy as np, csv
from qiskit_aer import StatevectorSimulator
import qiskit.qasm2, qiskit.qasm3
from mqt import ddsim

In [61]:
# @calculate_execution_time
def get_random_circ(n=None, d=None, g=None, h=None):
    qc, qr = random_circ_h_const(n, h, 0.2)
    # qc, qr = random_circ_g_const(n, d)
    return qc, qr

""" # Bell State circuit
qc = QuantumCircuit(2)
qc.h([0,1])
qc.cz(0,1)
qc.h(1)
d = qc.depth()
"""


' # Bell State circuit\nqc = QuantumCircuit(2)\nqc.h([0,1])\nqc.cz(0,1)\nqc.h(1)\nd = qc.depth()\n'

In [62]:
# @calculate_execution_time
def get_stvec_poly(qc, initial_state):
    terms, wire_array = create_poly(qc, n)
    ivs = [j[0] for j in wire_array]
    ovs = [j[-1] for j in wire_array]
    print("Input variables are: ", ivs)
    print("Output variables are: ", ovs)
    ttb = truthtable(terms, n, t, initial_state, ivs, np)
    return statevector_(ttb, n, t, ovs, np)
    # counts = {} # : To-Do

def get_stvec_ddsim(qc):
    backend = ddsim.DDSIMProvider().get_backend("statevector_simulator")
    job = backend.run(qc)
    result = job.result()
    return result.get_statevector()

def get_stvec_aer(qc):
    backend = StatevectorSimulator()
    res = backend.run(qc).result()
    return res.get_statevector()


In [63]:
def get_time_poly(qc, initial_state):
    # Time Calculation for Simulation using polynomial equation
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()

    state_vector = get_stvec_poly(qc, initial_state)
    print("Statevector using boolean equation: ")
    threshold = 1e-8
    for i, amp in enumerate(state_vector):
        if abs(amp) > threshold:
            print(f"|{i:0{n}b}> : {amp:.6f}")

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return state_vector, cpu_time, wall_time

def get_time_ddsim(qc):
    # Time Calculation for Simulation using DDSIM by MQT
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()

    state_vector = get_stvec_ddsim(qc)

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return state_vector, cpu_time, wall_time

def get_time_aer(qc):
    # Time Calculation for Simulation using Qiskit's Aer Simulator
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()

    state_vector = get_stvec_aer(qc)
    # printing the statevector amplitudes with a threshold
    print("Statevector using aer simulator: ")
    threshold = 1e-8
    for i, amp in enumerate(state_vector):
        if abs(amp) > threshold:
            print(f"|{i:0{qc.num_qubits}b}> : {amp:.6f}")

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return state_vector, cpu_time, wall_time


In [67]:
data = []
stVectors = []
for n in range(7,8):
    for h in range(6,7):
        # print(f"Value of h: {h}")
        qc, qr = get_random_circ(n=n, h=h)
        print(qc)
        n = qc.width()
        h = list(instruction.operation.name for index,
                instruction in enumerate(qc.data)).count('h')
        d = qc.depth()
        g = gate_counts(qc)
        t = n+h
        # print(f'Value of n: {n}, d: {d}, g: {g}, h: {h}')

        # on variables x0 x1 x2 x3 .... continuosly upto n
        # initial_state = [random.randint(0, 1) for _ in range(n)]
        initial_state = [0 for _ in range(n)]
        # print(f'Initial state of qubits is: {initial_state}')

        stvec_poly, cpu_time_poly, wall_time_poly = get_time_poly(qc, initial_state)
        stvec_ddsim, cpu_time_ddsim, wall_time_ddsim = get_time_ddsim(qc)
        stvec_aer, cpu_time_aer, wall_time_aer = get_time_aer(qc)

        stVectors.append([stvec_poly, stvec_ddsim, stvec_aer])
        data.append([n, h, d, g, t,
                    cpu_time_poly, wall_time_poly,
                    cpu_time_ddsim, wall_time_ddsim,
                    cpu_time_aer, wall_time_aer
                    ])
        # Store the circuit in QASM2 and QASM3 format
        qc_qasm2 = qiskit.qasm2.dumps(qc)
        qc_qasm3 = qiskit.qasm3.dumps(qc)
        with open('run1\qc_qasm2.qasm', 'a') as file:
            file.write(f"\n\n// Circuit: n={n}, h={h}, d={d}, g={g}\n")
            file.write(qc_qasm2)
        with open('run1\qc_qasm3.qasm', 'a') as file:
            file.write(f"\n\n// Circuit: n={n}, h={h}, d={d}, g={g}\n")
            file.write(qc_qasm3)
        
with open('run1\program_data.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    # writer.writerow(['n', 'h', 'd', 'g', 't',
    #                     'cpu_time_poly', 'wall_time_poly',
    #                     'cpu_time_ddsim', 'wall_time_ddsim',
    #                     'cpu_time_aer', 'wall_time_aer'
    #                     ])
    writer.writerows(data)

                                ┌───┐┌───┐            ░                    »
q_0: ─■───────────────────────■─┤ Z ├┤ Z ├─■───────■──░───────────────■──■─»
      │                       │ └───┘└───┘ │       │  ░    ┌───┐      │  │ »
q_1: ─■───■────────■──■───────■────────────┼───────┼──░──■─┤ Z ├──────┼──┼─»
      │   │  ┌───┐ │  │       │            │ ┌───┐ │  ░  │ ├───┤      │  │ »
q_2: ─┼───■──┤ Z ├─■──┼───────┼───■────■───┼─┤ H ├─┼──░──┼─┤ H ├──────┼──┼─»
      │      └───┘ │  │ ┌───┐ │   │    │   │ └───┘ │  ░  │ ├───┤      │  │ »
q_3: ─┼────────────┼──■─┤ Z ├─┼───┼────■───┼───────┼──░──┼─┤ Z ├──────┼──┼─»
      │ ┌───┐      │  │ ├───┤ │   │    │   │       │  ░  │ └───┘      │  │ »
q_4: ─┼─┤ Z ├──────■──■─┤ Z ├─■───■────■───┼───────┼──░──┼───■────────■──┼─»
      │ └───┘           └───┘     │        │       │  ░  │   │  ┌───┐ │  │ »
q_5: ─┼───────────────────────────■────────■───────■──░──■───┼──┤ Z ├─┼──┼─»
      │ ┌───┐┌───┐                                    ░      │  └───┘ │  │ »

C:\Users\isams\AppData\Local\Temp\ipykernel_12092\1502391411.py:13: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = ddsim.DDSIMProvider().get_backend("statevector_simulator")
C:\Users\isams\AppData\Local\Temp\ipykernel_12092\4171144178.py:51: DeprecationWarning: The return type of saved statevectors has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Statevector` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  for i, amp in enumerate(state_vector):


In [13]:
n = 10
h = 15
qc, qr = get_random_circ(n=n, h=h)
n = qc.width()
h = list(instruction.operation.name for index,
        instruction in enumerate(qc.data)).count('h')
d = qc.depth()
g = gate_counts(qc)
t = n+h
initial_state = [0 for _ in range(n)]
print(gate_counts(qc))
print(qc.count_ops())
# qc.draw(fold=-1)

33
OrderedDict([('h', 15), ('barrier', 15), ('z', 1), ('cz', 1), ('ccz', 1)])


In [10]:
stvec_poly, cpu_time_poly, wall_time_poly = get_time_poly(qc, initial_state)
print(cpu_time_poly, wall_time_poly)

Input variables are:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Output variables are:  [21, 20, 2, 3, 24, 17, 23, 10, 22, 9]
state vector using boolean equation: 
 [0. 0. 0. ... 0. 0. 0.]
3.421875 3.493514060974121


In [38]:
# # Load circuits from QASM file
# filename = "multiple_circuits.qasm"
# circuits = []

# with open(filename, 'r') as file:
#     qasm_code = file.read()

# # Split QASM code based on markers
# circuit_codes = qasm_code.split("//")

# for code in circuit_codes:
#     code = code.strip()
#     if code:
#         circuit = QuantumCircuit.from_qasm_str(code)
#         circuits.append(circuit)

# Now `circuits` list contains all the reconstructed quantum circuits